# Model 8.2. Neural Network (TensorFlow/Keras)

## Importing libraries

In [2]:
### Importing the libraries
import numpy as np
import pandas as pd
#from sklearnex import patch_sklearn

#patch_sklearn()

#plotting lib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


#Sklearn Lib metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, KFold

# Pipelines : 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config

#GradientBoostingClassifier and AdaboostClassifier
import tensorflow as tf
from tensorflow import keras

#Missing values : 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
import missingno as msno

#Dummy
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split


pd.set_option("display.max_columns",None)

## Importing and cleaning the data : 

In [261]:
train_df = pd.read_csv("Data/train.csv")
test_df = pd.read_csv("Data/test.csv")
#removing id for train_df
train_df.drop("id", axis = 1, inplace = True)
test_id = test_df[["id"]]
test_df.drop("id", axis = 1, inplace = True)

In [4]:
# Transforming object into categories 
# for train
for i in train_df.columns:
    if train_df[i].dtypes == "object":
        train_df[i] = train_df[i].astype("category")
# For test
for i in test_df.columns:
    if test_df[i].dtypes == "object":
        test_df[i] = test_df[i].astype("category")

In [10]:
X = train_df.drop("high_income", axis = 1)
y = train_df["high_income"]

# Grouping our features that are categories in one vector
# Same for numeric

categorical_features = [i for i in X.columns if X[i].dtype.name == "category"]
numerical_features = [i for i in X.columns if X[i].dtype.name != "category"]

## Splitting the data into a train and a validation set

In [143]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2, random_state=1, stratify=y)
y_train

2110     0
44057    0
37761    0
44201    1
31675    0
        ..
30863    0
20221    1
35500    0
40565    0
44910    0
Name: high_income, Length: 43685, dtype: int64

## Creating a pipeline in order to process the data 

In [144]:
#Pipeline to process the data before 
numeric_transformer = Pipeline(
    steps=[("imputer_num", SimpleImputer(strategy="mean")),
           ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[("imputer_cat", SimpleImputer(strategy="constant", fill_value="Missing")),
           
           ("encoder" , OneHotEncoder(handle_unknown="ignore", sparse=False))]
)



preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(
    steps=[("preprocessor", preprocessor)]
)


set_config(display="diagram")
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['occ_code_level2', 'age',
                                                   'stock_dividends',
                                                   'wage_per_hour',
                                                   'capital_losses',
                                                   'own_or_self',
                                                   'ind_code_level2',
                                                   'capital_gains',
                                                   'weeks_worked', 'num_emp',
                                                   'vet_benefits']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer_ca...
                                                   'hisp_origin', 'education',
                                                   'vet_question',
                                                   'country_self',
                                                   'mig_move_reg', 'hs_college',
                                                   'class_worker', 'mig_same',
                                                   'unemp_reason',
                                                   'state_prev_res', 'race',
                                                   'country_mother', 'sex',
                                                   'ind_code_level1',
                                                   'citizenship',
                                                   'union_member',
                                                   'fam_under_18',
                                                   'marital_stat',
                                                   'region_prev_res',
                                                   'mig_chg_reg',
                                                   'country_father',
                                                   'occ_code_level1',
                                                   'full_or_part_emp',
                                                   'det_hh_fam_stat'])]))])

In [264]:
new_x_train = clf.fit_transform(X_train)
new_x_valid = clf.transform(X_valid)
new_x_test = clf.transform(test_df)
new_x_train

array([[-8.27689714e-01, -2.02540599e+00, -1.73594115e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.31482797e-01,  3.95946317e-01, -1.73594115e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.31482797e-01,  0.00000000e+00, -2.38474242e-17, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       ...,
       [-8.27689714e-01,  0.00000000e+00, -1.73594115e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.27689714e-01,  1.22430368e+00, -1.73594115e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 9.03620732e-01,  0.00000000e+00, -1.73594115e-01, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [146]:
print(new_x_train.shape)
print(new_x_valid.shape)

(43685, 408)
(10922, 408)


## Running some Neural net models using Keras

In [147]:
model = keras.models.Sequential([
    keras.layers.Dense(units=300, activation="relu"),
    keras.layers.Dense(units=100, activation="relu"),
    keras.layers.Dense(units=2,  activation="softmax")
])

model.compile(loss = "sparse_categorical_crossentropy", # as y is encoded as a vector of labels and not as a one hot matrix
              optimizer = "sgd",
              metrics = ["accuracy"])


In [148]:
history = model.fit(x=new_x_train, y=y_train,
                    validation_data=(new_x_valid, y_valid),
                    batch_size = 64,
                    epochs=20)

Epoch 1/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3902 - accuracy: 0.8138 - val_loss: 0.3476 - val_accuracy: 0.8390
Epoch 2/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3296 - accuracy: 0.8476 - val_loss: 0.3318 - val_accuracy: 0.8474
Epoch 3/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3187 - accuracy: 0.8526 - val_loss: 0.3254 - val_accuracy: 0.8512
Epoch 4/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3136 - accuracy: 0.8563 - val_loss: 0.3223 - val_accuracy: 0.8526
Epoch 5/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3098 - accuracy: 0.8582 - val_loss: 0.3227 - val_accuracy: 0.8527
Epoch 6/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3074 - accuracy: 0.8594 - val_loss: 0.3240 - val_accuracy: 0.8528
Epoch 7/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3053 - accuracy: 0.8597 - val_loss: 0.3178 - val_accuracy: 0.8559
Epoch 

## Another attempt : 

In [149]:
model = keras.models.Sequential([
    keras.layers.Dense(units=300, activation="relu"),
    keras.layers.Dense(units=50, activation="relu"),
    keras.layers.Dense(units=2,  activation="softmax")
])

model.compile(loss = "sparse_categorical_crossentropy", # as y is encoded as a vector of labels and not as a one hot matrix
              optimizer = "sgd",
              metrics = ["accuracy"])
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=1)


In [150]:
history = model.fit(x=new_x_train, y=y_train,
                    validation_data=(new_x_valid, y_valid),
                    batch_size = 64,
                    epochs=20,callbacks=[early_stopping_cb])

Epoch 1/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3899 - accuracy: 0.8141 - val_loss: 0.3459 - val_accuracy: 0.8392
Epoch 2/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3280 - accuracy: 0.8477 - val_loss: 0.3314 - val_accuracy: 0.8471
Epoch 3/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3181 - accuracy: 0.8537 - val_loss: 0.3268 - val_accuracy: 0.8513
Epoch 4/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3124 - accuracy: 0.8570 - val_loss: 0.3218 - val_accuracy: 0.8537
Epoch 5/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3090 - accuracy: 0.8578 - val_loss: 0.3197 - val_accuracy: 0.8535
Epoch 6/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3065 - accuracy: 0.8603 - val_loss: 0.3243 - val_accuracy: 0.8550
Epoch 7/20
683/683 [==============================] - 2s 3ms/step - loss: 0.3042 - accuracy: 0.8610 - val_loss: 0.3180 - val_accuracy: 0.8552
Epoch 

In [151]:
y_pred = np.argmax(model.predict(new_x_valid), axis = 1)

342/342 [==============================] - 1s 2ms/step


In [152]:
y_pred

array([1, 0, 0, ..., 0, 0, 1])

In [153]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid, y_pred)


0.8597326496978576

# New test by changing the learning rate : 

In [256]:

tf.random.set_seed(1)


from keras.optimizers import Adam
model = keras.models.Sequential([
    keras.layers.Dense(units=300, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=300, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=50, activation="relu"),
    keras.layers.Dense(units=2,  activation="softmax")
    
])



model.compile(Adam(lr=0.001),loss = "sparse_categorical_crossentropy", # as y is encoded as a vector of labels and not as a one hot matrix
              
              metrics = ["accuracy"])
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, verbose=1)

history = model.fit(x=new_x_train, y=y_train,
                    validation_data=(new_x_valid, y_valid),
                    batch_size = 64,
                    epochs=20,callbacks=[early_stopping_cb])


/Users/adrien/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/20
683/683 [==============================] - 10s 13ms/step - loss: 0.3291 - accuracy: 0.8462 - val_loss: 0.3147 - val_accuracy: 0.8551
Epoch 2/20
683/683 [==============================] - 10s 14ms/step - loss: 0.3069 - accuracy: 0.8604 - val_loss: 0.3172 - val_accuracy: 0.8556
Epoch 3/20
683/683 [==============================] - 11s 16ms/step - loss: 0.2967 - accuracy: 0.8670 - val_loss: 0.3114 - val_accuracy: 0.8583
Epoch 4/20
683/683 [==============================] - 13s 20ms/step - loss: 0.2899 - accuracy: 0.8698 - val_loss: 0.3112 - val_accuracy: 0.8602
Epoch 5/20
683/683 [==============================] - 15s 21ms/step - loss: 0.2810 - accuracy: 0.8739 - val_loss: 0.3104 - val_accuracy: 0.8620
Epoch 6/20
683/683 [==============================] - 12s 18ms/step - loss: 0.2723 - accuracy: 0.8788 - val_loss: 0.3232 - val_accuracy: 0.8598
Epoch 7/20
683/683 [==============================] - 12s 17ms/step - loss: 0.2655 - accuracy: 0.8818 - val_loss: 0.3292 - val_accuracy:

In [257]:
y_pred = np.argmax(model.predict(new_x_valid), axis = 1)
from sklearn.metrics import accuracy_score
accuracy_score(y_valid, y_pred)

342/342 [==============================] - 2s 4ms/step


0.8620216077641457

64 mini batch, 300 --> 0.2 --> 300 --> 0.2 --> 50 -->2 = 0.8620216077641457 best so far ! 

## Trying to publish this model

In [265]:
NN_keras = test_id
y_pred = np.argmax(model.predict(new_x_test), axis = 1)
NN_keras["high_income"] = y_pred
NN_keras


190/190 [==============================] - 1s 6ms/step


/var/folders/r3/t8xp3_0d2qg9znqckxzmdc2w0000gp/T/ipykernel_34611/2369048823.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NN_keras["high_income"] = y_pred


,id,high_income
0,1,0
1,2,0
2,3,0
3,4,1
4,5,0
...,...,...
6063,6064,0
6064,6065,1
6065,6066,0
6066,6067,0


In [266]:
NN_keras.to_csv("Predictions/NN_keras.csv",index = False, header=True)